In [1]:
import os, sys
import django
PROJECTPATH = '/mnt/c/Users/liath1/coding/kodtjanst/'
sys.path.insert(0, PROJECTPATH)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings.local")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # https://docs.djangoproject.com/en/4.1/topics/async/#async-safety
os.chdir(PROJECTPATH)
django.setup()

In [4]:
from kodtjanst.models import Kodverk, CodeableConceptAttributes

import pandas as pd
from pdb import set_trace
from django.core.exceptions import MultipleObjectsReturned, ObjectDoesNotExist
import requests
from requests.exceptions import ConnectionError

In [8]:
for record in CodeableConceptAttributes.objects.all():
    if (record.källa is not None) and ('länk' in record.källa):
        url = record.källa.split(' ')[0].split('=')[1]
        try:
            request = requests.get(url)
        except ConnectionError as e:
            print(url)
        
        if request.status_code == 404:
            print(record.kodverk_from.titel_på_kodverk, url, request.status_code)

HSA Innehåll Verksamhetskod https://inera.atlassian.net/wiki/download/attachments/397444985/hsa_innehall_verksamhetskod_version_5.15_2022-04-20.pdf 404
VGRKV_bokningstyp https://www.hl7.org/fhir/v2/0276/index.html 404
ICD-10-SE https://www.socialstyrelsen.se/utveckla-verksamhet/e-halsa/klassificering-och-koder/icd-10/ 404


In [ ]:
new_term_list = []
for index, row in df.iterrows():
    try:
        if Begrepp.objects.get(term=row.term):
            continue            
    except ObjectDoesNotExist as e:
        print(e, row.Term)
        new_term = Begrepp(
            term = row.term,
            definition = row.definition,
            källa = 'GetLaboratoryOrderOutcomes version 4',
            #anmärkningar = row.Anmärkningar,
            status = row.status
            beställare = Bestallare.objects.get(id=row.bestallare)
            )
        new_term_list.append(new_term)
#Begrepp.objects.bulk_create(new_term_list)

In [ ]:
new_synonym_list = []
for index, row in df.iterrows():
    if (row.Synonym is not None) and (pd.isna(row.Synonym) == False):
        try:
            Begrepp.objects.get(term=row.Term)
        except MultipleObjectsReturned as e:
            print(e)
            set_trace()
        new_synonym_list.append(Synonym(
                                        begrepp_id = Begrepp.objects.get(term=row.Term).id,
                                        synonym = row.Synonym,
                                        synonym_status = 'Beslutad'
                                    )
                               )
        

In [ ]:
#Synonym.objects.bulk_create(new_synonym_list)